In [2]:
import sys
import signal

# Windows does not have SIGKILL
if sys.platform == "win32":
    signal.SIGKILL = signal.SIGTERM  # fallback

In [10]:
import nemo.collections.asr as nemo_asr
import torch

# Select processing device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load Typhoon ASR Real-Time model
print("Loading Typhoon ASR Real-Time...")
asr_model = nemo_asr.models.ASRModel.from_pretrained(
    model_name="scb10x/typhoon-asr-realtime",
    map_location=device
)

Using device: cpu
Loading Typhoon ASR Real-Time...


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /scb10x/typhoon-asr-realtime/resolve/main/typhoon-asr-realtime.nemo (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))"), '(Request ID: 7ebbe0dc-e37d-450d-9a35-58a159c28a19)')' thrown while requesting HEAD https://huggingface.co/scb10x/typhoon-asr-realtime/resolve/main/typhoon-asr-realtime.nemo
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /scb10x/typhoon-asr-realtime/resolve/main/typhoon-asr-realtime.nemo (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))"), '(Request ID: ebbe6a43-fadf-4588-9156-4d81ef70a5a7)')' thrown while requesting HEAD https://huggingface.co/s

SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /scb10x/typhoon-asr-realtime/resolve/main/typhoon-asr-realtime.nemo (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))"), '(Request ID: 1b107397-c8bf-424c-ba72-9a91f1cd9d9c)')

In [4]:
import os
import sys
import time
from pathlib import Path
import nemo.collections.asr as nemo_asr
import torch
import librosa
import soundfile as sf

In [5]:
def prepare_audio(input_path, target_sr=16000):
    input_path = Path(input_path)
    if not input_path.exists():
        raise FileNotFoundError(f"File not found: {input_path}")

    supported_formats = ['.wav', '.mp3', '.m4a', '.flac', '.ogg', '.aac', '.webm']
    if input_path.suffix.lower() not in supported_formats:
        raise ValueError(f"Unsupported format: {input_path.suffix}. Supported formats are: {supported_formats}")

    processed_path = f"processed_{input_path.stem}.wav"

    y, sr = librosa.load(str(input_path), sr=None)
    if y is None:
        raise IOError("Failed to load audio file.")

    duration = len(y) / sr

    if sr != target_sr:
        y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)

    y = y / (max(abs(y)) + 1e-8)
    sf.write(processed_path, y, target_sr)
    
    return processed_path, duration

In [8]:
processed_path, duration = prepare_audio(input_path='audio/recorded_audio.wav')

In [9]:
processed_path

'processed_recorded_audio.wav'

In [ ]:
import nemo.collections.asr as nemo_asr
import torch
import sounddevice as sd
import numpy as np

# เลือก device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# โหลด model
asr_model = nemo_asr.models.ASRModel.from_pretrained(
    model_name="scb10x/typhoon-asr-realtime",
    map_location=device
)

# ตั้งค่าการบันทึกเสียง
sample_rate = 16000  # Typhoon model ใช้ 16kHz
block_duration = 1.0  # 1 วินาทีต่อ batch
block_size = int(sample_rate * block_duration)

print("เริ่มฟังเสียงจากไมโครโฟน... กด Ctrl+C เพื่อหยุด")

def callback(indata, frames, time, status):
    if status:
        print(status)
    audio_data = indata[:, 0]  # ใช้ channel แรก
    audio_tensor = torch.tensor(audio_data, dtype=torch.float32).unsqueeze(0).to(device)
    
    # Transcribe
    with torch.no_grad():
        transcription = asr_model.transcribe(audio_tensor)
        print(">>", transcription)

# เริ่ม stream
with sd.InputStream(channels=1, samplerate=sample_rate, blocksize=block_size, callback=callback):
    while True:
        pass
